In [5]:
import cv2,os

data_path = r"C:\Users\HI\Downloads\face_mask\dataset\dataset"
categories = os.listdir(data_path)
labels = list(range(len(categories)))

label_dict = dict(zip(categories, labels))

print("Label dictionary:", label_dict)
print("Categories:", categories)
print("Labels:", labels)


Label dictionary: {'with mask': 0, 'without mask': 1}
Categories: ['with mask', 'without mask']
Labels: [0, 1]


In [6]:
img_size=100
data=[]
target=[]

for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            #Coverting the image into gray scale
            resized=cv2.resize(gray,(img_size,img_size))
            #resizing the gray scale into 100x100, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)

        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

In [7]:
pip install tensorflow

  Using cached numpy-2.1.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.1.3-cp310-cp310-win_amd64.whl (12.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
  You can safely remove it manually.


In [10]:
import numpy as np
from tensorflow.keras.utils import to_categorical

data = np.array(data) / 255.0
data = np.reshape(data, (data.shape[0], img_size, img_size, 1))
target = np.array(target)

new_target = to_categorical(target)


In [11]:
np.save('data',data)
np.save('target',new_target)

In [12]:
#loading the save numpy arrays in the previous code
data=np.load('data.npy')
target=np.load('target.npy')

In [13]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

c:\Users\HI\Documents\face_mask\.conda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

# 1. Load your data
data = np.load('data.npy')        # e.g. shape (samples, 100, 100, 1)
target = np.load('target.npy')    # e.g. shape (samples, 4) OR (samples, 2, 2)

print("Data shape:", data.shape)
print("Target shape:", target.shape)

# 2. If target is 3D like (samples, 2, 2), flatten it to 2D (samples, 4)
if len(target.shape) == 3:
    target = target.reshape(target.shape[0], -1)  # flatten last dims
    print("Flattened target shape:", target.shape)

# 3. Split data into train and test sets
train_data, test_data, train_target, test_target = train_test_split(
    data, target, test_size=0.1, random_state=42
)

# 4. Build your model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=data.shape[1:]),
    MaxPooling2D(),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(train_target.shape[1], activation='softmax')  # match output units with target shape
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# 5. Setup ModelCheckpoint callback (must save as '.keras' or '.h5')
checkpoint = ModelCheckpoint(
    'model-{epoch:03d}.keras',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='auto'
)

# 6. Train the model
history = model.fit(
    train_data,
    train_target,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[checkpoint]
)

# 7. Evaluate on test set
test_loss, test_acc = model.evaluate(test_data, test_target)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_acc}")


ImportError: DLL load failed while importing lapack_lite: The specified module could not be found.

In [7]:
from keras.models import load_model
import cv2
import numpy as np

In [8]:
model = load_model('model-026.keras')

face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)

labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

while(True):
    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)

    for x,y,w,h in faces:
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)
        confidence = np.max(result)
        label = np.argmax(result)
        if confidence < 0.7:
            label_text = 'Uncertain'
            color = (255, 255, 0)
        else:
            label_text = labels_dict[label]
            color = color_dict[label]
        cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
        cv2.putText(img,label_text,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,2)

    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    if(key==27):
        break

cv2.destroyAllWindows()
source.release()


In [9]:
import os

print(os.path.exists('haarcascade_frontalface_default.xml'))  # Should print True


True


In [10]:
face_clsfr = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


In [ ]:
while True:
    ret, img = source.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_clsfr.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_img = gray[y:y+h, x:x+w]  # use grayscale image
        resized = cv2.resize(face_img, (100, 100))
        normalized = resized / 255.0
        reshaped = np.reshape(normalized, (1, 100, 100, 1))  # note last dim=1
        result = model.predict(reshaped)

        label = np.argmax(result, axis=1)[0]

        cv2.rectangle(img, (x, y), (x+w, y+h), color_dict[label], 2)
        cv2.rectangle(img, (x, y-40), (x+w, y), color_dict[label], -1)
        cv2.putText(img, labels_dict[label], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.imshow('LIVE', img)
    key = cv2.waitKey(1)

    if key == 27:
        break

cv2.destroyAllWindows()
source.release()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━